## QQQ vs QLD

Monthly Averaging

### Library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
from datetime import date
from datetime import timedelta
import random
import calendar
import requests
from azure.storage.blob import BlobServiceClient
import plotly.graph_objects as go
import json

### Global Variables

In [3]:
Symbols = []

In [4]:
Months = {
    1 : 'Jan',
    2 : 'Feb',
    3 : 'Mar',
    4 : 'Apr',
    5 : 'May',
    6 : 'Jun',
    7 : 'Jul',
    8 : 'Aug',
    9 : 'Sep',
    10 : 'Oct',
    11 : 'Nov',
    12 : 'Dec',
    'Jan' : 1,
    'Feb' : 2,
    'Mar' : 3,
    'Apr' : 4,
    'May' : 5,
    'Jun' : 6,
    'Jul' : 7,
    'Aug' : 8,
    'Sep' : 9,
    'Oct' : 10,
    'Nov' : 11,
    'Dec' : 12
}

### Helper Functions

In [5]:
# only works for yyyy-mm-dd
def StringToDate(string, seperation_character='-'):
    date_list = string.split(seperation_character)
    return(date(int(date_list[0]), int(date_list[1]), int(date_list[2])))

In [6]:
def add_month(number_of_months, month, year):
    if month + number_of_months > 12 :
        return (month + number_of_months) % 12, year + (month + number_of_months) // 12
    else:
        return month + number_of_months, year

In [7]:
def GetPrevDates(inDate, number_of_previous_dates):
    prevDates = []
    i = 1
    while(i <= number_of_previous_dates):
        prevDates.append(inDate - timedelta(days=i))
        i+=1
    return prevDates

In [8]:
def GetPrevMonths(inDate, number_of_previous_months, flag = 1):
    prevMonths = []
    i = 1
    year = inDate.year
    month = inDate.month

    while(i <= number_of_previous_months):
        month = month - 1
        if month < 1:
            month += 12
            year -=1
        prevMonths.append((year, month))
        i+=1
    
    if flag == 1: 
        return prevMonths
    else:
        return prevMonths[len(prevMonths) - 1]

In [9]:
#### To write logs into ADLS
def write_dataframe_to_datalake(df, StockSymbol="admin"):
    STORAGEACCOUNTURL = "https://adlszeus.blob.core.windows.net"
    STORAGEACCOUNTKEY = "ksL9a2OZFCiKFYPn6hzTNJcY4WI2Nq2xSsRlUD8cDH3dBBEvePAhJqErSP6QKN27so/2ayW3DnO7O8s4uPtUZA=="
    CONTAINERNAME = "price-prediction"
    # BLOBNAME = "AzureFunctionLogs/temp1.csv"
    BLOBNAME = "MaxStockStore" + "\\" + StockSymbol + "_" + ".json"

    blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

    blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)

    blob_client_instance.upload_blob(data=df.to_json(orient='records'))

    return True

In [10]:
#### To write logs into ADLS
def read_dataframe_from_datalake(StockSymbol="admin"):
    STORAGEACCOUNTURL = "https://adlszeus.blob.core.windows.net"
    STORAGEACCOUNTKEY = "ksL9a2OZFCiKFYPn6hzTNJcY4WI2Nq2xSsRlUD8cDH3dBBEvePAhJqErSP6QKN27so/2ayW3DnO7O8s4uPtUZA=="
    CONTAINERNAME = "price-prediction"
    # BLOBNAME = "AzureFunctionLogs/temp1.csv"
    BLOBNAME = "MaxStockStore" + "\\" + StockSymbol + "_" + ".json"

    blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

    blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)

    # blob_client_instance.upload_blob(data=df.to_json(orient='records'))
    in_file = blob_client_instance.download_blob()

    return pd.read_json(in_file, orient="records") # type: ignore
# 

In [11]:
def get_data(StockSymbol):
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={StockSymbol}&outputsize=full&apikey=DHJWDUYNRKAIHAED'
    r = requests.get(url)
    data = r.json()

    data_df = pd.DataFrame.from_dict(data['Time Series (Daily)']).transpose().reset_index()
    data_df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Dividend Amount', 'Split Coefficient']
    data_df = data_df[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    data_df['StockSymbol'] = StockSymbol
    return data_df

### Functions

In [12]:
def CalculatePosition_BuyLastDayOfMonth(prev_n_months, input_df, inDate=date.today()):
    # initial variables
    number_of_stocks = 0
    amount_invested = 0

    Bought_at_df = pd.DataFrame()

    stocks_bought_at = GetPrevMonths(inDate=inDate, number_of_previous_months=prev_n_months)

    current_value = input_df[input_df['Date'] == inDate]['Close'].to_list()

    if len(current_value) == 0:
        current_value = input_df.sort_values(by='Date')[input_df['Date'] < inDate].tail(1)['Close'].to_list()
        if len(current_value) == 0:
            return

    for month in stocks_bought_at:
        last_day = calendar.monthrange(month[0], month[1])[1]


        if len(input_df[input_df['Date'] == date(month[0], month[1], last_day)]['Close'].values) == 0:
            selected_row_s = input_df.sort_values(by='Date')[input_df['Date'] < date(month[0], month[1], last_day)].tail(1)
            amount_invested += selected_row_s['Close'].values[0] * 100
            selected_row_s['Amount_Invested'] = selected_row_s['Close'].values[0] * 100
            selected_row_s['Current_Value'] = current_value[0] * 100
            selected_row_s['Mark_to_Market'] = current_value[0] * 100 - selected_row_s['Close'].values[0] * 100
            selected_row_s['%'] = round((current_value[0] * 100 - selected_row_s['Close'].values[0] * 100) * 100 / (selected_row_s['Close'].values[0] * 100), 2)

            Bought_at_df = pd.concat([Bought_at_df, selected_row_s], ignore_index=True)
            # print("if", selected_row_s['Date'])
            # Bought_at.append(selected_row_s.to_dict(), ignore_index=True)
            # print(qqq_df.sort_values(by='Date')[qqq_df['Date'] < date(month[0], month[1], last_day)].head(1)['Close'].values)

        else:
            selected_row_s = input_df[input_df['Date'] == date(month[0], month[1], last_day)]
            amount_invested += selected_row_s['Close'].values[0] * 100
            selected_row_s['Amount_Invested'] = selected_row_s['Close'].values[0] * 100
            selected_row_s['Current_Value'] = current_value[0] * 100
            selected_row_s['Mark_to_Market'] = current_value[0] * 100 - selected_row_s['Close'].values[0] * 100
            selected_row_s['%'] = round((current_value[0] * 100 - selected_row_s['Close'].values[0] * 100) * 100 / (selected_row_s['Close'].values[0] * 100), 2)


            Bought_at_df = pd.concat([Bought_at_df, selected_row_s], ignore_index=True)
            # print("else", selected_row_s['Date'])
            # Bought_at.append(selected_row_s.to_dict(), ignore_index=True)
            # print(qqq_df[qqq_df['Date'] == date(month[0], month[1], last_day)]['Close'].values)

        
        number_of_stocks += 100 


    try:
        return current_value[0] * number_of_stocks, amount_invested, current_value[0] * number_of_stocks - amount_invested, Bought_at_df
    except:
        return (0, 0, 0)


In [13]:
# 0 for Peak / 1 for Dip
# Also checks for % higher or lower condition
def CheckPeakOrDip(past_val, current_val, future_val, amount_per_stock, percent=10, flag=0):
    # Peak
    if flag == 0:
        if current_val > past_val and current_val > future_val and (current_val - amount_per_stock) * 100/amount_per_stock > percent:
            return True
        else:
            return False
    # Dip
    elif flag == 1:
        if current_val < past_val and current_val < future_val  and current_val < amount_per_stock*(1 - (percent/100)):
            # print("dip", current_val)
            return True
        else:   
            return False

In [14]:
def CalculatePosition_PeaksAndDips(prev_n_months, input_df, inDate=date.today(), initial_number_of_stocks = 100):
    stocks_bought_at = GetPrevMonths(inDate=inDate, number_of_previous_months=prev_n_months, flag=0)

    investment_date = date(int(stocks_bought_at[0]), int(stocks_bought_at[1]), int(inDate.day + 1))

    target_df = input_df[input_df['Date'] >= investment_date]
    target_df = target_df[input_df['Date'] <= inDate]

    target_df['Avg'] = target_df.apply(lambda row: (row['High'] + row['Low']) / 2, axis=1)

    target = target_df.sort_values(by=['Date']).to_dict(orient='records')

    # initial variables
    output = []
    number_of_stocks = 0
    initial_amount_invested = None

    amount_invested = 0
    amount_per_stock = 0
    amount_in_hand = 0

    past_val = None
    current_val = None
    future_val = None
    # Flag for Buy or Sell of Stocks
    # 0 - Bought - Check for peak and Sell
    # 1 - Sold - Check for dip and Buy
    BuySell = 0

    for data_point in target:
        #Setup
        if initial_amount_invested == None:
            # Initial Buy - Buys a set number of initial stocks
            initial_amount_invested = data_point['Avg'] * initial_number_of_stocks
            amount_invested = data_point['Avg'] * initial_number_of_stocks
            amount_per_stock = data_point['Avg']
            amount_in_hand = 0
            number_of_stocks = initial_number_of_stocks

            past_val = data_point['Avg']

            # print(f"amount_invested : {amount_invested}, amount_per_stock : {amount_per_stock}, amount_in_hand : {amount_in_hand}, number_of_stocks : {number_of_stocks}")
            temp = data_point
            temp['Amount_Invested'] = amount_invested
            temp['Amount_Per_Stock'] = amount_per_stock
            temp['Amount_In_Hand'] = amount_in_hand
            temp['Number_Of_Stocks_Bought'] = number_of_stocks
            temp['Mark_To_Market'] = amount_invested + amount_in_hand - initial_amount_invested
            temp['%'] = (amount_invested + amount_in_hand - initial_amount_invested) * 100 / initial_amount_invested

            output.append(temp)

            # set flag to detect peak
            BuySell = 0 
            continue

        elif current_val == None:
            current_val = data_point['Avg']
            continue

        elif future_val == None:
            future_val = data_point['Avg']

            # Checking for Peak or Dip
            if CheckPeakOrDip(past_val=past_val, current_val=current_val, future_val=future_val, amount_per_stock=amount_per_stock, flag=BuySell):
                # selling
                if BuySell == 0:
                    amount_invested = 0
                    amount_per_stock = future_val
                    amount_in_hand += number_of_stocks * future_val
                    number_of_stocks = 0
                    BuySell = 1
                    # print(f"amount_invested : {amount_invested}, amount_per_stock : {amount_per_stock}, amount_in_hand : {amount_in_hand}, number_of_stocks : {number_of_stocks}, BuySell : {BuySell}")

                    temp = data_point
                    temp['Amount_Invested'] = amount_invested
                    temp['Amount_Per_Stock'] = amount_per_stock
                    temp['Amount_In_Hand'] = amount_in_hand
                    temp['Number_Of_Stocks_Bought'] = number_of_stocks
                    temp['Mark_To_Market'] = amount_invested + amount_in_hand - initial_amount_invested
                    temp['%'] = (amount_invested + amount_in_hand - initial_amount_invested) * 100 / initial_amount_invested

                    output.append(temp)
                # buying 
                elif BuySell == 1:
                    number_of_stocks = int(amount_in_hand // future_val)
                    amount_invested += number_of_stocks * future_val
                    amount_per_stock = future_val
                    amount_in_hand -= amount_invested
                    BuySell = 0
                    # print(f"amount_invested : {amount_invested}, amount_per_stock : {amount_per_stock}, amount_in_hand : {amount_in_hand}, number_of_stocks : {number_of_stocks}, BuySell : {BuySell}")

                    temp = data_point
                    temp['Amount_Invested'] = amount_invested
                    temp['Amount_Per_Stock'] = amount_per_stock
                    temp['Amount_In_Hand'] = amount_in_hand
                    temp['Number_Of_Stocks_Bought'] = number_of_stocks
                    temp['Mark_To_Market'] = amount_invested + amount_in_hand - initial_amount_invested
                    temp['%'] = (amount_invested + amount_in_hand - initial_amount_invested) * 100 / initial_amount_invested

                    output.append(temp)
                
                past_val = current_val
                current_val = future_val
                future_val = None
            
            else:
                past_val = current_val
                current_val = future_val
                future_val = None
                    
    return initial_amount_invested, pd.DataFrame.from_records(output)

Time Horizon - Holding Period

Dips and Peaks - % movement

Stock Symbol - Multiple Inputs

Investment Amount=


Do the Same for Plug Power and Tesla with an investment of 25K and 2 Years horizon with % of 5
API get data for max time period

In [15]:

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=IBM&outputsize=full&apikey=demo'
r = requests.get(url)
data = r.json()

In [16]:
data['Time Series (Daily)']

{'2023-07-12': {'1. open': '135.07',
  '2. high': '135.33',
  '3. low': '132.575',
  '4. close': '132.84',
  '5. adjusted close': '132.84',
  '6. volume': '3732189',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-07-11': {'1. open': '133.66',
  '2. high': '134.56',
  '3. low': '133.23',
  '4. close': '134.44',
  '5. adjusted close': '134.44',
  '6. volume': '2925238',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-07-10': {'1. open': '131.76',
  '2. high': '133.05',
  '3. low': '131.695',
  '4. close': '132.9',
  '5. adjusted close': '132.9',
  '6. volume': '2369425',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-07-07': {'1. open': '131.78',
  '2. high': '133.855',
  '3. low': '131.75',
  '4. close': '132.08',
  '5. adjusted close': '132.08',
  '6. volume': '2982738',
  '7. dividend amount': '0.0000',
  '8. split coefficient': '1.0'},
 '2023-07-06': {'1. open': '133.235',
  '2. high': '133.9',
  '3. l

### Global Variables

In [17]:
qqq_df = pd.read_csv("Data\\QQQ_max.csv")
qld_df = pd.read_csv("Data\\QLD_max.csv")

In [18]:
qqq_df['Date'] = qqq_df.apply(lambda row: StringToDate(row['Date']), axis=1)
qld_df['Date'] = qld_df.apply(lambda row: StringToDate(row['Date']), axis=1)

### Process

In [19]:
qqq_df['StockSymbol'] =  'QQQ'
qld_df['StockSymbol'] =  'QLD'

In [20]:
combined_df = pd.concat([qqq_df, qld_df], ignore_index=True)

In [21]:
combined_df.to_csv('Data\\Combined.csv')

In [22]:
fig = px.line(
    qqq_df,
    x="Date",
    y="Close",
    # size="Population",
    color="StockSymbol",
    hover_name="StockSymbol",
    # log_x=True,
    # size_max=60,
)

In [23]:
fig.show()

In [24]:
import plotly.graph_objects as go

In [25]:
temp_df = combined_df[combined_df['StockName'] == 'QQQ'].head(50)

KeyError: 'StockName'

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=temp_df['Date'],
                open=temp_df['Open'],
                high=temp_df['High'],
                low=temp_df['Low'],
                close=temp_df['Close'])],
                )

fig.show()

In [ ]:
df = pd.read_csv("Data\\QQQ_max.csv") # replace with your own data source
candle_fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])],
                )

In [ ]:
candle_fig.update_yaxes()

In [ ]:
stocksymbol = 'RIVN'

In [ ]:
data_df = get_data(StockSymbol=stocksymbol)

In [ ]:
data_df

,Date,Open,High,Low,Close,Adj Close,Volume,StockSymbol
0,2023-07-07,22.92,25.63,22.6,24.7,24.7,230871351,RIVN
1,2023-07-06,20.05,21.7699,19.64,21.62,21.62,99871584,RIVN
2,2023-07-05,20.705,21.05,19.435,20.43,20.43,118923443,RIVN
3,2023-07-03,17.84,19.7,17.8,19.56,19.56,79984819,RIVN
4,2023-06-30,16.22,16.75,15.78,16.66,16.66,40422801,RIVN
...,...,...,...,...,...,...,...,...
410,2021-11-16,163.8,179.4699,153.78,172.01,172.01,94036577,RIVN
411,2021-11-15,130.8,152.53,127.51,149.36,149.36,64982343,RIVN
412,2021-11-12,128.645,135.2,125.25,129.95,129.95,50345677,RIVN
413,2021-11-11,114.625,125.0,108.01,122.99,122.99,83668202,RIVN


In [ ]:
write_dataframe_to_datalake(data_df, StockSymbol=stocksymbol)

True

In [ ]:
def CalculatePosition_BuyLastDayOfMonth(prev_n_months, input_df, amount=1000, inDate=date.today()):
    # initial variables
    number_of_stocks = 0
    amount_invested = 0

    Bought_at_df = pd.DataFrame()

    if amount < 0:
        return (0, 0, 0, Bought_at_df)


    stocks_bought_at = GetPrevMonths(inDate=inDate, number_of_previous_months=prev_n_months)

    current_value = input_df[input_df['Date'] == inDate]['Close'].to_list()

    if len(current_value) == 0:
        current_value = input_df.sort_values(by='Date')[input_df['Date'] < inDate].tail(1)['Close'].to_list()
        if len(current_value) == 0:
            return

    for month in stocks_bought_at:
        last_day = calendar.monthrange(month[0], month[1])[1]
        number_of_stocks_bought = 0

        if len(input_df[input_df['Date'] == date(month[0], month[1], last_day)]['Close'].values) == 0:
            selected_row_s = input_df.sort_values(by='Date')[input_df['Date'] < date(month[0], month[1], last_day)].tail(1)

            number_of_stocks_bought = amount // selected_row_s['Close'].values[0]

            amount_invested += selected_row_s['Close'].values[0] * number_of_stocks_bought
            selected_row_s['Number_of_Stocks_Bought'] = number_of_stocks_bought

            selected_row_s['Amount_Invested'] = round(selected_row_s['Close'].values[0] * number_of_stocks_bought, 2)
            selected_row_s['Current_Value'] = round(current_value[0] * number_of_stocks_bought, 2)


            selected_row_s['Mark_to_Market'] = round(current_value[0] * number_of_stocks_bought - selected_row_s['Close'].values[0] * number_of_stocks_bought,2)
            selected_row_s['%'] = round((current_value[0] * number_of_stocks_bought - selected_row_s['Close'].values[0] * number_of_stocks_bought) * 100 / (selected_row_s['Close'].values[0] * number_of_stocks_bought), 2)

            Bought_at_df = pd.concat([Bought_at_df, selected_row_s], ignore_index=True)

        else:
            selected_row_s = input_df[input_df['Date'] == date(month[0], month[1], last_day)]

            number_of_stocks_bought = amount // selected_row_s['Close'].values[0]

            amount_invested += selected_row_s['Close'].values[0] * number_of_stocks_bought
            selected_row_s['Number_of_Stocks_Bought'] = number_of_stocks_bought

            selected_row_s['Amount_Invested'] = round(selected_row_s['Close'].values[0] * number_of_stocks_bought, 2)
            selected_row_s['Current_Value'] = round(current_value[0] * number_of_stocks_bought, 2)

            selected_row_s['Mark_to_Market'] = round(current_value[0] * number_of_stocks_bought - selected_row_s['Close'].values[0] * number_of_stocks_bought, 2)
            selected_row_s['%'] = round((current_value[0] * number_of_stocks_bought - selected_row_s['Close'].values[0] * number_of_stocks_bought) * 100 / (selected_row_s['Close'].values[0] * number_of_stocks_bought), 2)


            Bought_at_df = pd.concat([Bought_at_df, selected_row_s], ignore_index=True)
        
        number_of_stocks += number_of_stocks_bought 


    try:
        return current_value[0] * number_of_stocks, amount_invested, current_value[0] * number_of_stocks - amount_invested, Bought_at_df
    except:
        return (0, 0, 0, Bought_at_df)

In [ ]:
#### To write logs into ADLS
def read_dataframe_from_datalake(StockSymbol="admin"):
    STORAGEACCOUNTURL = "https://adlszeus.blob.core.windows.net"
    STORAGEACCOUNTKEY = "ksL9a2OZFCiKFYPn6hzTNJcY4WI2Nq2xSsRlUD8cDH3dBBEvePAhJqErSP6QKN27so/2ayW3DnO7O8s4uPtUZA=="
    CONTAINERNAME = "price-prediction"
    # BLOBNAME = "AzureFunctionLogs/temp1.csv"
    BLOBNAME = "MaxStockStore" + "\\" + StockSymbol + "_" + ".json"

    blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)

    blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)

    # blob_client_instance.upload_blob(data=df.to_json(orient='records'))
    in_file = blob_client_instance.download_blob()

    return pd.read_json(in_file, orient="records") # type: ignore

In [ ]:
input_df = read_dataframe_from_datalake("QQQ")

In [ ]:
returns, invested, profit, details_df = CalculatePosition_BuyLastDayOfMonth(3, input_df, amount=10000, inDate=np.datetime64(date.today(), 'D'))

AttributeError: 'numpy.datetime64' object has no attribute 'year'

In [ ]:
x = np.datetime64(date.today(), 'D')

In [ ]:
pd.Timestamp(x).to_pydatetime().date()

datetime.date(2023, 7, 10)

In [ ]:
def npDatetimeToDate(inDate):
    return pd.Timestamp(inDate).to_pydatetime().date()

In [ ]:
npDatetimeToDate(x)

datetime.date(2023, 7, 10)

In [ ]:
pd.Timestamp.now()

Timestamp('2023-07-10 13:31:01.494039')

In [45]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
np.random.seed(1)

N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N) + 5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N) - 5

symbols = 'circle-dot'

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='lines',
                    name='lines'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                     mode='lines+markers',
#                     name='lines+markers'))
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='markers', name='markers', marker_symbol=symbols,
                           marker_line_width=2, marker_size=10))

fig.show()

In [33]:
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator

raw_symbols = SymbolValidator().values
namestems = []
namevariants = []
symbols = []
for i in range(0,len(raw_symbols),3):
    name = raw_symbols[i+2]
    symbols.append(raw_symbols[i])
    namestems.append(name.replace("-open", "").replace("-dot", ""))
    namevariants.append(name[len(namestems[-1]):])


fig = go.Figure(go.Scatter(mode="markers", x=namevariants, y=namestems, marker_symbol=symbols,
                           marker_line_color="midnightblue", marker_color="lightskyblue",
                           marker_line_width=2, marker_size=15,
                           hovertemplate="name: %{y}%{x}<br>number: %{marker.symbol}<extra></extra>"))
fig.update_layout(title="Mouse over symbols for name & number!",
                  xaxis_range=[-1,4], yaxis_range=[len(set(namestems)),-1],
                  margin=dict(b=0,r=0), xaxis_side="top", height=1400, width=400)
fig.show()

In [ ]:
import plotly

In [ ]:
plotly.plot('graph', [{
  mode: 'text',
  x: [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,1,2,3,4,5,6,7,8,9,
      10,11,12,13,14,15,15,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,15,1],
  y: [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,
      2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4],
  text: ['μ', '°', '±', 'Ͷ', 'Θ','ϕ','‣','	⃤','ℹ','ↆ','↔',
         '∇','⌖','①','⑳','↹','◈','☂','★','✅', '✪','➤','⤨','⬗','🎮',
         '💗','🜲','🟀','','','','▶','P','⚠️','ℹ️','😅','⚽'],
  hovertemplate: 'This is you tooltip',
  type: 'scatter',
  textfont: {
    size: 25,
    color: 'black'
  }
}])


NameError: name 'mode' is not defined

In [55]:
qqq_df.reset_index(drop=True)

,Date,Open,High,Low,Close,Adj Close,Volume,StockSymbol
0,1999-03-10,51.125000,51.156250,50.281250,51.062500,43.838398,5232000,QQQ
1,1999-03-11,51.437500,51.734375,50.312500,51.312500,44.053043,9688600,QQQ
2,1999-03-12,51.125000,51.156250,49.656250,50.062500,42.979862,8743600,QQQ
3,1999-03-15,50.437500,51.562500,49.906250,51.500000,44.214005,6369000,QQQ
4,1999-03-16,51.718750,52.156250,51.156250,51.937500,44.589588,4905800,QQQ
...,...,...,...,...,...,...,...,...
6102,2023-06-08,349.119995,353.619995,348.890015,353.149994,353.149994,47153900,QQQ
6103,2023-06-09,354.630005,357.660004,353.029999,354.500000,354.500000,53155900,QQQ
6104,2023-06-12,356.190002,360.589996,355.200012,360.489990,360.489990,44606500,QQQ
6105,2023-06-13,363.290009,364.109985,360.029999,363.260010,363.260010,57097800,QQQ
